# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

Предоставлены данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных уже выполнена.

Построить модель с максимально большим значением *accuracy*. Необходимо довести долю правильных ответов по крайней мере до 0.75. Проверить *accuracy* на тестовой выборке.

<b> Описание данных </b>

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Описание хода работы

1. Изучить файла с данными 
Путь к файлу: /datasets/users_behavior.csv. Скачать датасет
2. Разделить исходные данные на обучающую, валидационную и тестовую выборки.
3. Исследовать качество разных моделей, меняя гиперпараметры. Кратко написать выводы исследования.
4. Проверить качество модели на тестовой выборке.

## Настройки рабочей тетради

In [1]:
# Импорт библиотек

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import os

In [2]:
# Настройки библиотек

 
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)

In [3]:
# Путь к директории с данными

path = 'datasets/'

## Изучение файла с данными

In [4]:
df = pd.read_csv(os.path.join(path, 'users_behavior.csv'))

df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
#Проверка на дубликаты и пропуски

print('Количество дубликатов:', df.duplicated().sum())
print('Количество пропущенных значений:')
print(df.isna().sum())

Количество дубликатов: 0
Количество пропущенных значений:
calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64


<b>Вывод</b>

Исходные данные представлены в удобном для дальнейшей работы варианте. Дубликатов и пропусков нет.

## Разделение данных на выборки

Исходные данные представлены одним датасетом, его будем разбивать на обучающую, валидационную и тестовую выборки в соотношении 60/20/20 соответственно.

In [6]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                              test_size=0.4, random_state=12345)
features_test, features_valid, target_test, target_valid = train_test_split(features_valid, target_valid, 
                                                                              test_size=0.5, random_state=12345)

## Исследование моделей

Поставленная задача - задача классификации. Для решения такой задачи нам известны 3 вида моделей: дерево решений, случайный лес, логистическая регрессия. Проверим их все с разными гиперпараметрами для выявления лучшей модели.

In [7]:
#Дерево решений

best_model_tree = None
best_result_tree = 0
best_depth_tree = 0
for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predictions = model_tree.predict(features_valid)
    result = accuracy_score(target_valid, predictions)
    if result > best_result_tree:
        best_model_tree = model_tree
        best_result_tree = result
        best_depth = depth
        
print('Accuracy лучшей модели дерева решений:', result, 'с глубиной дерева равной', best_depth)

Accuracy лучшей модели дерева решений: 0.7884914463452566 с глубиной дерева равной 7


In [8]:
#Случайный лес

best_model_forest = None
best_result_forest = 0
best_depth_forest = 0
best_est_forest = 0
for est in range(10, 101, 10):
    for depth in range(1, 11):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_forest.fit(features_train, target_train)
        result = model_forest.score(features_valid, target_valid)
        if result > best_result_forest:
            best_best_model_forestmodel = model_forest
            best_result_forest = result
            best_depth_forest = depth
            best_est_forest = est
            
print('Accuracy наилучшей модели случайного леса на валидационной выборке:', best_result_forest, 
      'Количество деревьев:', best_est_forest, 'Глубина дерева:', best_depth_forest)

Accuracy наилучшей модели случайного леса на валидационной выборке: 0.8133748055987559 Количество деревьев: 10 Глубина дерева: 9


In [9]:
#Логистичская регрессия

model_log = LogisticRegression(random_state=12345, solver='liblinear')
model_log.fit(features_train, target_train) # обучите модель на тренировочной выборке
result = model_log.score(features_valid, target_valid) # получите метрику качества модели на валидационной выборке

print("Accuracy модели логистической регрессии:", result)

Accuracy модели логистической регрессии: 0.7402799377916018


<b>Вывод</b>

Рассмотрено 3 вида моделей с разными значениями гиперпараметров. Модель логистической регрессии не подходит нам вообще, так как её accuracy меньше 0.75. 

Наилучшие модели дерева решений и случайного леса дают результат accuracy больше 0.75. По условии поставленной задачи необходимо "Постройте модель с максимально большим значением accuracy." значит итоговой моделью станет модель случайного леса с количеством деревьев 10 и глубиной дерева 9

## Проверка модели на тестовой выборке

In [10]:
model = RandomForestClassifier(random_state=12345, n_estimators=best_est_forest, max_depth=best_depth_forest)
model.fit(features_train, target_train)

predictions = model.predict(features_test)
result = accuracy_score(target_test, predictions)
print('Accuracy итоговой модели:', result)

Accuracy итоговой модели: 0.7853810264385692


<b>Вывод</b> 

Выбранная модель выдаёт необходимое значение accuracy на тестовой выборке.